# Introduction to using FHIR Data Pipes and FHIR Views
In this example we will show how to use FHIR Views with data generated using the FHIR Data Pipes library

# Prerequisites:

**Note**: All commands need to be run from the root directory of the fhir-data-pipes repo

1. Bring up a Hive ThriftServer:
    ```
    docker-compose  -f ./docker/compose-controller-spark-sql-single.yaml up --build  --force-recreate -d
    ```

2. Bring up a HAPI FHIR server:
    ```
    docker-compose  -f ./docker/hapi-compose.yml up --force-recreate -d
    ```

3. Load data into the HAPI FHIR server:
    ```
    python3 ./synthea-hiv/uploader/main.py HAPI http://localhost:8091/fhir --input_dir ./synthea-hiv/sample_data
    ```

4. In the FHIR Pipelines Control Panel page, http://localhost:8090, click on “Run Full” button to export the data to SQL-on-FHIR format. The exported parquet files will be stored under the `docker/dwh` directory


Install the FHIR Views library:

In [15]:
!pip3 install google-fhir-views[r4,spark]

Defaulting to user installation because normal site-packages is not writeable


In [16]:
import pandas

from sqlalchemy import dialects
from sqlalchemy import engine

from google.fhir.views import r4
from google.fhir.views import spark_runner

# The Spark dataset containing FHIR data. This may be read-only to the user.
fhir_dataset = "default"

# The Spark dataset where we will create views, value sets, and other derived tables
# as needed. This must be writeable by the user. This will use the default project
# where this notebook is running.
analysis_dataset = "statin_analysis_example"

dialects.registry.register("hive", "pyhive.sqlalchemy_hive", "HiveDialect")

# The endpoint of the Hive ThriftServer to connect to
query_engine = engine.create_engine("hive://localhost:10001/default")

# Create a runner to execute the views over Spark.
runner = spark_runner.SparkRunner(
    query_engine=query_engine,
    fhir_dataset=fhir_dataset,
    view_dataset=analysis_dataset,
    snake_case_resource_tables=True,
)

The cell below loads the parquet files we've downloaded into our Spark ThriftServer:

In [17]:
destination_directory_path = "/dwh/controller_*"
with query_engine.connect() as curs:
    curs.execute(f"DROP TABLE IF EXISTS {fhir_dataset}.encounter;")
    curs.execute(f"DROP TABLE IF EXISTS {fhir_dataset}.observation;")
    curs.execute(f"DROP TABLE IF EXISTS {fhir_dataset}.patient;")
    curs.execute(
        f"CREATE TABLE IF NOT EXISTS {fhir_dataset}.encounter USING"
        f" PARQUET LOCATION '{destination_directory_path}/Encounter/*.parquet';"
    )
    curs.execute(
        f"CREATE TABLE IF NOT EXISTS {fhir_dataset}.observation USING PARQUET"
        f" LOCATION '{destination_directory_path}/Observation/*.parquet';"
    )
    curs.execute(
        f"CREATE TABLE IF NOT EXISTS {fhir_dataset}.patient USING PARQUET"
        f" LOCATION '{destination_directory_path}/Patient/*.parquet';"
    )

    curs.execute(f"DROP DATABASE IF EXISTS {analysis_dataset} CASCADE;")
    curs.execute(f"CREATE DATABASE IF NOT EXISTS {analysis_dataset};")

# Explore the data

Getting a general feeling for the data is an important first step in any analysis. So let's create some views of FHIR resources and take a look at common fields.



In [18]:
# Load views based on the base FHIR R4 profile definitions.
views = r4.base_r4()

# Create shorthand names for resources we will work with.
conds = views.view_of("Condition")
meds = views.view_of("MedicationRequest")
obs = views.view_of("Observation")
pats = views.view_of("Patient")

Getting a general feeling for the data is an important first step in any analysis. So let's create some views of FHIR resources and take a look at common fields. Note that the results of this notebook are very similar to the BigQuery notebook. Being able to define views independent of the runner is one of the main advantages of using FHIR Views.

In [19]:
runner.to_dataframe(
    obs.select(
        {
            "category": obs.category.coding.code,
            "code_display": obs.code.coding.display,
            "status": obs.status,
        }
    ),
    limit=5,
)

,category,code_display,status
0,"[""survey""]","[""CURRENT ANTIRETROVIRAL DRUGS USED FOR TREATM...",final
1,"[""survey""]","[""LOINC Code""]",final
2,"[""survey""]","[""TESTS ORDERED""]",final
3,"[""survey""]","[""TESTS ORDERED""]",final
4,"[""survey""]","[""TUBERCULOSIS PROPHYLAXIS PLAN""]",final


Of course just a sample of codes isn't too useful -- we really want a summary of what codes exist in a field and how many there are. Fortunately, the Spark runner supports a summarize_codes method that accepts a view and a field name and does exactly that.

In [20]:
runner.summarize_codes(obs, obs.category)

,system,code,display,count
0,http://terminology.hl7.org/CodeSystem/observat...,survey,survey,675
1,http://terminology.hl7.org/CodeSystem/observat...,exam,exam,13


# Creating Views

FHIR Data is complicated, but there is usually a flat, tabular form of it that can satisfy a given use case and data set.

For example, imagine we need a simple table of patients with their current address. This isn't trivial to query since the address is a nested repeated field. Fortunately we can build a FHIRPath expression to find the current address and create a flattened view using that.

(This can vary by dataset, but in this case we determine the current address by finding the first address that does not have a period attached to it.)

In [21]:
# For this dataset we interpret the current address as one where period is empty.

current = pats.address.where(pats.address.period.empty())

simple_pats = pats.select(
    {
        "id": pats.id,
        "gender": pats.gender,
        "birthdate": pats.birthDate,
        "street": current.line,
        "city": current.city,
        "state": current.state,
        "zip": current.postalCode,
    }
)

runner.to_dataframe(simple_pats, limit=5)

,id,gender,birthdate,street,city,state,zip
0,4765,male,1984-09-09,"[""754 Feil Tunnel Unit 36""]","[""Springfield""]","[""MA""]","[""01119""]"
1,4767,female,1999-04-26,"[""766 King Landing Suite 14""]","[""Spencer""]","[""MA""]","[""01562""]"
2,4768,female,1999-02-12,"[""508 Auer Lodge""]","[""Wilbraham""]","[""MA""]",[]
3,4770,male,1999-07-01,"[""230 Olson Fort Suite 50""]","[""Dennis""]","[""MA""]",[]
4,4776,female,1993-07-31,"[""612 Kuhlman Skyway Apt 45""]","[""Amherst""]","[""MA""]",[]


That's nice, but suppose we want to create it as an actual Spark view -- basically a virtual table that can be easily used by any application that uses Spark. We can simply turn that definition into a Spark view

In [22]:
runner.create_database_view(simple_pats, "patients")

In [23]:
pandas.read_sql_query(
    sql="SELECT * FROM statin_analysis_example.patients LIMIT 5",
    con=query_engine,
)

,id,gender,birthdate,street,city,state,zip
0,4765,male,1984-09-09,"[""754 Feil Tunnel Unit 36""]","[""Springfield""]","[""MA""]","[""01119""]"
1,4767,female,1999-04-26,"[""766 King Landing Suite 14""]","[""Spencer""]","[""MA""]","[""01562""]"
2,4768,female,1999-02-12,"[""508 Auer Lodge""]","[""Wilbraham""]","[""MA""]",[]
3,4770,male,1999-07-01,"[""230 Olson Fort Suite 50""]","[""Dennis""]","[""MA""]",[]
4,4776,female,1993-07-31,"[""612 Kuhlman Skyway Apt 45""]","[""Amherst""]","[""MA""]",[]


Now we have a nice, flattened patients table that meets the needs of our system and data.